<h1 style="background-color:black; font-family:'Brush Script MT',cursive;color:#03e8fc;font-size:200%; text-align:center;border-radius: 50% 20% / 10% 40%">Survival and the Appearance of Tumor in MRI</h1>

"MRI Manifestions Correlate with Survival of Glioblastoma Multiforme Patients"

Authors: Wen-bin Li, Kai Tang, Qian Chen, Shuai Li, Xiao-guang Qiu, Shao-wu Li, and Tao Jiang

Cancer Biol Med. 2012 Jun; 9(2): 120–123. - doi: 10.3969/j.issn.2095-3941.2012.02.007

"Glioblastoma multiforme (GBM) is the most common primary brain tumor in adults, with an annual age adjusted incidence of 3.0 to 3.6 per 100,000. GBM is classified by the World Health Organization as a grade IV glioma. Treatment modes include surgery, radiotherapy, and chemotherapy."

"The survival time of patients with GBM is significantly shorter than that of patients with lower-grade gliomas. Even within the same grade, the survival time for patients with GBM may also be highly variable. For more than 20 years, magnetic resonance imaging (MRI) has been used in the clinical diagnosis of gliomas to serve as a guide for clinical treatment. Most diagnoses for patients with malignant gliomas are followed by MRI. Thus, determining the relationship between survival and the appearance of tumor in MRI is of importance."

"Imaging features such as degrees of contrasted area, edema surrounding the tumor, and intensity in T2-weighted imaging can be easily collected from routine scans."

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3643659/

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Install Necessary Packages</span></h1><br>


NOTE: In order to use plotly you will need to:

Install all necessary packages/extensions following these instructions.

Sign up for a free account and activate your key following these instructions (only read the section ‘Initialization for Online Plotting’).

In [ ]:
!pip install  chart_studio

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Fluid Attenuated Inversion Recovery (FLAIR)</span></h1><br>

"T2 Fluid-Attenuated Inversion Recovery Resection for Glioblastoma Involving Eloquent Brain Areas Facilitated Through Awake Craniotomy and Clinical Outcome"

Authors: MingLu; Zheng-haoFu; Xiao-junHe; Jian-kanLu; Xin-qingDeng; De-liuLin; You-mingGu; Yan-fengFan; Ming-yaoLai; JuanLi; Ming-mingYang; Zhong-pingChen

https://doi.org/10.1016/j.wneu.2019.12.130

"Despite evidence that a greater extent of resection (EOR) improves survival, the role of extended resection based on magnetic resonance imaging (MRI) fluid-attenuated inversion recovery (FLAIR) in the prognosis of glioblastoma (GBM) remains controversial. This study aims to investigate the role of additional resection of FLAIR-detected abnormalities and its influence on clinical outcomes of patients with GBM."

"The threshold for removal of FLAIR abnormalities affecting survival was determined to be 25%. The median OS and PFS were shorter in the group with FLAIR resection <25% compared with the group with FLAIR resection ≥25% (12 months vs. 26 months; P = 0.001 and 6 months vs. 15 months; P = 0.016, respectively). Univariate and multivariate analyses identified tumor location within or near the eloquent brain areas and the 25% threshold for FLAIR EOR as independent factors affecting OS (overall survival) and PFS (progression-free survival)."

"Identifying a feasible threshold for the resection of FLAIR (fluid-attenuated inversion recovery) abnormalities is valuable in improving the survival of patients with GBM (Glioblastoma Multiforme). Extended resection of GBM involving eloquent brain areas was safe when using a combination of AC and SF-guided surgery."

https://www.sciencedirect.com/science/article/abs/pii/S1878875019331626

In [ ]:
#Code by Md. Redwan Karim Sony https://www.kaggle.com/redwankarimsony/visualizing-and-analyzing-dicoms-in-python/notebook?select=train.csv

#common packages 
import numpy as np 
import os
import copy
from math import *
import matplotlib.pyplot as plt
from functools import reduce
from glob import glob

#reading in dicom files
import pydicom

#skimage image processing packages
from skimage import measure, morphology
from skimage.morphology import ball, binary_closing
from skimage.measure import label, regionprops

#scipy linear algebra functions 
from scipy.linalg import norm
import scipy.ndimage

#ipywidgets for some interactive plots
from ipywidgets.widgets import * 
import ipywidgets as widgets

#plotly 3D interactive graphs 
import plotly
from plotly.graph_objs import *
import chart_studio

In [ ]:
chart_studio.tools.set_credentials_file(username='redwankarimsony', api_key='aEbXWsleQv7PJrAtOkBk')
#set plotly credentials here 
#this allows you to send results to your account plotly.tools.set_credentials_file(username=your_username, api_key=your_key)

#Sorry Md. Redwan, I promise I'll make an account of my own if I intend to work again with Dicoms.

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Loading DICOM Data</span></h1><br>

Let's set the paths of the dicom files and then we will be able to have a look at them.

Instead of BraTS21ID (Brain Tumor Classification ID) I called them as Samples. 

In [ ]:
#Code by Md. Redwan Karim Sony https://www.kaggle.com/redwankarimsony/visualizing-and-analyzing-dicoms-in-python/notebook?select=train.csv

sample_id = '00003'
sample_folder = f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{sample_id}/'
data_paths = glob(sample_folder + '/*/*.dcm')

# Print out the first 5 file names to verify we're in the right folder.
print (f'Total of {len(data_paths)} DICOM images.\nFirst 5 filenames:' )
data_paths[:5]

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">T1 weighted image</span></h1><br>

By Dr Yahya Baba and Dr Jeremy Jones et al.

"T1 weighted image (also referred to as T1WI or the "spin-lattice" relaxation time) is one of the basic pulse sequences in MRI and demonstrates differences in the T1 relaxation times of tissues.

A T1WI relies upon the longitudinal relaxation of a tissue's net magnetization vector (NMV). Basically, spins aligned in an external field (B0) are put into the transverse plane by a radiofrequency (RF) pulse. They then slide back toward the original equilibrium of B0. Not all tissues return back to equilibrium in the same amount of time, and a tissue's T1 reflects the amount of time taken for its protons' spins to realign with the main magnetic field (B0).

T1 weighting tends to have short TE and TR times."

https://radiopaedia.org/articles/t1-weighted-image

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">T2 weighted image</span></h1><br>

By Dr Ammar Haouimi and Dr Jeremy Jones et al.

T2 weighted image (T2WI) is one of the basic pulse sequences on MRI. The sequence weighting highlights differences on the T2 relaxation time of tissues.

<font color="#0000FF">Summary</font>

repetition time (TR): long

echo time (TE): long

flip angle: less important than with T1 weighting

fat: intermediate-bright

fluid: bright

<font color="#0000FF">Physics</font>

A T2WI relies upon the transverse relaxation (also known as "spin-spin" relaxation) of the net magnetization vector (NMV). T2 weighting tends to require long TE and TR times.

https://radiopaedia.org/articles/t2-weighted-image

In [ ]:
#Code by Md. Redwan Karim Sony https://www.kaggle.com/redwankarimsony/visualizing-and-analyzing-dicoms-in-python/notebook?select=train.csv

def load_scan(paths):
    slices = [pydicom.read_file(path ) for path in paths]
    slices.sort(key = lambda x: int(x.InstanceNumber), reverse = True)
    try:
        slice_thickness = np.abs(slices[0].ImagePositionSample[2] - slices[1].ImagePositionSample[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    image = image.astype(np.int16)
    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Sex Differences in MRI-Based Metrics of Glioma Invasion</span></h1><br>

"Sex Differences in MRI-Based Metrics of Glioma Invasion and Brain Mechanics"

Authors: Barrett J. Anderies, Sara F. Yee, Pamela R. Jackson, Cassandra R. Rickertsen, Andrea J.,Hawkins-Daarud, Sandra K. Johnston, Kamala R. Clark-Swanson, Joseph M. Hoxworth,Yuan Le, Yuxiang Zhou, Kay M. Pepin, Susan C. Massey, Leland S. Hu, John R. Huston III, Kristin R. Swanson 

doi: https://doi.org/10.1101/2020.11.21.352724 November 22, 2020.

Abbreviations:

HGG = high grade gliomas, T1Gd = T1-weighted gadolinium contrasted, T2-FLAIR = T2 weighted fluid
attenuated inversion recovery, ROI = region of interest, CC = contralateral control, CE = contrast
enhancing, MRI = magnetic resonance imaging; MRE = magnetic resonance elastography; MNO =
Mathematical Neuro-Oncology Lab; PNT = Precision Neurotherapeutics Innovation Program, D = tumor
diffusiveness, ⍴ = tumor proliferation, D/⍴ = tumor invasiveness

"Understanding how the true underlying tumor invasion profile corresponds to radiographic imaging is
critical to providing informed clinical care. To date, such studies have been limited due to the difficulty
of acquiring tissue for histological analysis. MRI imaging is a proxy measurement of tumors as it
primarily resolves contrast (T1Gd) and interstitial fluid accumulation (T2/FLAIR), which are measures of
leaky vasculature and edema, respectively. These measures may be correlated with clinical pathology
but do not necessarily relate to the local chemical and physical microenvironment within which the
tumor cells proliferate and migrate."

"Clinical imaging remains the primary method of monitoring gliomas and is mostly interpreted with
respect to changes in size. Increasing any understanding of how imaging features correspond to tumor
characteristics can drastically influence how therapy is chosen and how response to therapy is
determined. Much remains to be done, but that work represents a first look at how similar imaging may
reflect different tumor invasion depending on the sex of the patient."

https://www.biorxiv.org/content/10.1101/2020.11.21.352724v2

https://www.biorxiv.org/content/10.1101/2020.11.21.352724v1.full.pdf

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSidNItQO2zsxd1oZ4Hwmy0vGM-su6fJLqZJZRLimrbEUpW9IcTwl1opbA_9YhkEsLquQ&usqp=CAU)biorxivorg

#Run the following code to extract DICOM pixels for each slice location and display a single slice:

In [ ]:
#Code by Md. Redwan Karim Sony https://www.kaggle.com/redwankarimsony/visualizing-and-analyzing-dicoms-in-python/notebook?select=train.csv

# set path and load files 
sample_dicom = load_scan(data_paths)
sample_pixels = get_pixels_hu(sample_dicom)
#sanity check
plt.imshow(sample_pixels[80], cmap=plt.cm.bone);

In [ ]:
import pandas as pd

train = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
train.head().style.set_properties(**{'background-color':'black',
                                     'color': '#03e8fc'})

In [ ]:
import seaborn as sns

train.corr()
plt.figure(figsize=(10,4))
sns.heatmap(train.corr(),annot=True,cmap='Blues')
plt.show()

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Image Processing</span></h1><br>

Let' s use some thresholding and morphological operations to segment just the brain from the rest of the scan:

In [ ]:
#Code by Md. Redwan Karim Sony https://www.kaggle.com/redwankarimsony/visualizing-and-analyzing-dicoms-in-python/notebook?select=train.csv

def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)
    counts = counts[vals != bg]
    vals = vals[vals != bg]
    if len(counts) > 0:
        return vals[np.argmax(counts)]
    else:
        return None
    
def segment_brain_mask(image, fill_brain_structures=True):
    # not actually binary, but 1 and 2. 
    # 0 is treated as background, which we do not want
    binary_image = np.array(image >= -700, dtype=np.int8)+1
    labels = measure.label(binary_image)
 
    # Pick the pixel in the very corner to determine which label is air.
    # Improvement: Pick multiple background labels from around the  patient
    # More resistant to “trays” on which the patient lays cutting the air around the person in half
    background_label = labels[0,0,0]
 
    # Fill the air around the person
    binary_image[background_label == labels] = 2
 
    # Method of filling the brain structures (that is superior to 
    # something like morphological closing)
    if fill_brain_structures:
        # For every slice we determine the largest solid structure
        for i, axial_slice in enumerate(binary_image):
            axial_slice = axial_slice - 1
            labeling = measure.label(axial_slice)
            l_max = largest_label_volume(labeling, bg=0)
 
            if l_max is not None: #This slice contains some lung
                binary_image[i][labeling != l_max] = 1
    binary_image -= 1 #Make the image actual binary
    binary_image = 1-binary_image # Invert it, lungs are now 1
 
    # Remove other air pockets inside body
    labels = measure.label(binary_image, background=0)
    l_max = largest_label_volume(labels, bg=0)
    if l_max is not None: # There are air pockets
        binary_image[labels != l_max] = 0
 
    return binary_image

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">MGMT promoter methylation</span></h1><br>

"MGMT promoter methylation in malignant gliomas: ready for personalized medicine?"

Citation: Weller, M., Stupp, R., Reifenberger, G. et al. MGMT promoter methylation in malignant gliomas: ready for personalized medicine?. Nat Rev Neurol 6, 39–51 (2010). https://doi.org/10.1038/nrneurol.2009.197

"The DNA repair enzyme O6-methylguanine-DNA methyltransferase (MGMT) antagonizes the genotoxic effects of alkylating agents. MGMT promoter methylation is the key mechanism of MGMT gene silencing and predicts a favorable outcome in patients with glioblastoma who are exposed to alkylating agent chemotherapy. This biomarker is on the verge of entering clinical decision-making and is currently used to stratify or even select glioblastoma patients for clinical trials. In other subtypes of glioma, such as anaplastic gliomas, the relevance of MGMT promoter methylation might extend beyond the prediction of chemosensitivity, and could reflect a distinct molecular profile."

<font color="#0000FF">Key Points</font>

MGMT (O6-methylguanine-DNA methyltransferase) promoter methylation has become the most powerful molecular prognosticator in malignant gliomas.

MGMT promoter methylation is predictive for response to alkylating agent chemotherapy in glioblastoma.

Methylation-specific PCR is the only validated technique to derive prognostic information from determination of the MGMT status.

The MGMT status has become a parameter for stratification of patients with glioma within clinical trials.

https://www.nature.com/articles/nrneurol.2009.197

By running the code below, you are using skimage functions from above to create a mask that covers the brain. Redwan used both fill_brain_structures=True and fill_brain_structures=False, to isolate the brain and the internal structures. Let’s run the code below, and display an example of isolating the brain from the rest of the scan:

In [ ]:
#Code by Md. Redwan Karim Sony https://www.kaggle.com/redwankarimsony/visualizing-and-analyzing-dicoms-in-python/notebook?select=train.csv

# get masks 
segmented_brains = segment_brain_mask(sample_pixels, fill_brain_structures=False)
segmented_brains_fill = segment_brain_mask(sample_pixels, fill_brain_structures=True)
internal_structures = segmented_brains_fill - segmented_brains

# isolate brain from the rest of the scan
copied_pixels = copy.deepcopy(sample_pixels)
for i, mask in enumerate(segmented_brains_fill): 
    get_high_vals = mask == 0
    copied_pixels[i][get_high_vals] = 0
seg_brain_pixels = copied_pixels
# sanity check
f, ax = plt.subplots(1,2, figsize=(10,6))
ax[0].imshow(sample_pixels[80], cmap=plt.cm.bone)
ax[0].axis(False)
ax[0].set_title('Original')
ax[1].imshow(seg_brain_pixels[80], cmap=plt.cm.bone)
ax[1].axis(False)
ax[1].set_title('Segmented')
plt.show()

#What did I make wrong? I tried to isolate the brain from the rest of the scan. But I failed.

#If anybody knows how to fix that, please tell me. I will fail in all the segmentation below.

Don’t worry about the masks yet, Redwan will show you some visualizations later on.

I'm worried because I failed to isolate the brain. Read the Original code to learn:

https://www.kaggle.com/redwankarimsony/visualizing-and-analyzing-dicoms-in-python/notebook?select=train.csv

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">2D Visualizations Techniques</span></h1><br>


Non Interactive: When visualizing data, Redwan found very beneficial to visualize each process of your script. This will not only help you understand each step of your code, but it makes for a very nice, clean presentation. The code below will, in essence, visually convey a story on how you segmented your data:

Original image

The binary mask that covers the brain

Highlighting internal structures of the brain using one of the masks

Extracting internal structures using GK clustering

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">MGMT Promoter Methylation and Gene Expression Levels in Glioblastoma</span></h1><br>

"Correlation of MGMT promoter methylation status with gene and protein expression levels in glioblastoma"

Authors: Miyuki Uno, Sueli Mieko Oba-Shinjo, Anamaria Aranha Camargo, Ricardo Pereira Moura, Paulo Henrique de Aguiar, Hector Navarro Cabrera, Marcos Begnami, Sérgio Rosemberg, Manoel Jacobsen Teixeira, Suely Kazue Nagahashi Marie

Clinical Sciences,  Clinics 66 (10),  2011  https://doi.org/10.1590/S1807-59322011001000013

"RESULTS: MGMT promoter methylation was found in 43.1% of glioblastoma by methylation-specific PCR and 38.8% by pyrosequencing. A low level of MGMT gene expression was correlated with positive MGMT promoter methylation (p = 0.001)."

"However, no correlation was found between promoter methylation and MGMT protein expression (p = 0.297). The mean survival time of glioblastoma patients submitted to adjuvant therapy was significantly higher among patients with MGMT promoter methylation (log rank = 0.025 by methylation-specific PCR and 0.004 by pyrosequencing), and methylation was an independent predictive factor that was associated with improved prognosis by multivariate analysis."

"DISCUSSION AND CONCLUSION: MGMT promoter methylation status was a more reliable predictor of susceptibility to adjuvant therapy and prognosis of glioblastoma than were MGMT protein or gene expression levels. Methylation-specific polymerase chain reaction and pyrosequencing methods were both sensitive methods for determining MGMT promoter methylation status using DNA extracted from frozen tissue."

https://www.scielo.br/j/clin/a/tzYgxYpz9b6Y54ftcD7bJ6F/?lang=en

In [ ]:
#Code by Md. Redwan Karim Sony https://www.kaggle.com/redwankarimsony/visualizing-and-analyzing-dicoms-in-python/notebook?select=train.csv

f, ax = plt.subplots(2,2, figsize = (10,10))

# pick random slice 
slice_id = 80

ax[0,0].imshow(sample_pixels[slice_id], cmap=plt.cm.bone)
ax[0,0].set_title('Original Dicom')
ax[0,0].axis(False)


ax[0,1].imshow(segmented_brains_fill[slice_id], cmap=plt.cm.bone)
ax[0,1].set_title('Brain Mask')
ax[0,1].axis(False)

ax[1,0].imshow(seg_brain_pixels[slice_id], cmap=plt.cm.bone)
ax[1,0].set_title('Segmented Brain')
ax[1,0].axis(False)

ax[1,1].imshow(seg_brain_pixels[slice_id], cmap=plt.cm.bone)
ax[1,1].imshow(internal_structures[slice_id], cmap='jet', alpha=0.7)
ax[1,1].set_title('Segmentation with \nInternal Structure')
ax[1,1].axis(False);

#Even copying the script, I ruined it! 

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Interactive</span></h1><br>

There are really only a few ways to display multiple images on jupyter notebook — manually plot each image one by one, or make a plot with multiple columns and rows to display in one figure. Unfortunately, this is not helpful for those that want to scan through each image and get a better understanding of the data. With the code below you create an interactive slide bar that lets you scroll through the images (But the catch is that you have to run the code in **interactive window**. So if you want to use the slider window to browse through the slices, fork the code and run it manually in interactive mode):

#Below there is a slide bar that didn't slide. Hence, I tried to make another slide. 

Inside the 2nd slide bar slides. In the viewer the image disappears. No show with ipywidgets.

In [ ]:
#Code by Md. Redwan Karim Sony https://www.kaggle.com/redwankarimsony/visualizing-and-analyzing-dicoms-in-python/notebook?select=train.csv

# slide through dicom images using a slide bar #It doesn't Slide! 
#plt.figure(1)
def dicom_animation(x):
    plt.imshow(sample_pixels[x], cmap = plt.cm.gray)
    return x
interact(dicom_animation, x=(0, len(sample_pixels)-1));

In [ ]:
#Code by Jose Carmona @josecarmona

%matplotlib widget

from ipywidgets import interact

def dicom_animation(x):
    plt.imshow(sample_pixels[x], cmap = plt.cm.gray)
    plt.plot()

interact(dicom_animation, x=(0, len(sample_pixels)-1))

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Interactive 2</span></h1><br>

"Another way to visualize the MRI scans in a bit lively fashion is to use gif images. Basically GIFs are a series of images shown at an preconfigured interval automatically. However, the upper side of GIF is that, it stores images in lossless compression. </blue>. So it is highly efficent and accurate for storing MRI scan slides."

In [ ]:
#Code by Md. Redwan Karim Sony https://www.kaggle.com/redwankarimsony/visualizing-and-analyzing-dicoms-in-python/notebook?select=train.csv

import imageio
from IPython import display
print('Original Image Slices before processing')
imageio.mimsave(f'./{sample_id}.gif', sample_pixels, duration=0.1)
display.Image(f'./{sample_id}.gif', format='png')

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Prediction of malignant glioma</span></h1><br>

"Prediction of malignant glioma grades using contrast-enhanced T1-weighted and T2-weighted magnetic resonance images based on a radiomic analysis"

Citation: Nakamoto, T., Takahashi, W., Haga, A. et al. Prediction of malignant glioma grades using contrast-enhanced T1-weighted and T2-weighted magnetic resonance images based on a radiomic analysis. Sci Rep 9, 19411 (2019). https://doi.org/10.1038/s41598-019-55922-0


"The authors conducted a feasibility study to predict malignant glioma grades via radiomic analysis using contrast-enhanced T1-weighted magnetic resonance images (CE-T1WIs) and T2-weighted magnetic resonance images (T2WIs). They proposed a framework and applied it to CE-T1WIs and T2WIs (with tumor region data) acquired preoperatively from 157 patients with malignant glioma (grade III: 55, grade IV: 102) as the primary dataset and 67 patients with malignant glioma (grade III: 22, grade IV: 45) as the validation dataset."

"Radiomic features such as size/shape, intensity, histogram, and texture features were extracted from the tumor regions on the CE-T1WIs and T2WIs."

"The Wilcoxon–Mann–Whitney (WMW) test and least absolute shrinkage and selection operator logistic regression (LASSO-LR) were employed to select the radiomic features. Various machine learning (ML) algorithms were used to construct prediction models for the malignant glioma grades using the selected radiomic features."

"The prediction models were constructed using the LR, a support vector machine (SVM), a standard neural network (SNN), a random forest (RF), and a Naïve Bayes (NB)." 

"Leave-one-out cross-validation (LOOCV) was implemented to evaluate the performance of the prediction models in the primary dataset. The selected radiomic features for all folds in the LOOCV of the primary dataset were used to perform an independent validation. As evaluation indices, accuracies, sensitivities, specificities, and values for the area under receiver operating characteristic curve (or simply the area under the curve (AUC)) for all prediction models were calculated."

"The mean AUC value for all prediction models constructed by the ML algorithms in the LOOCV of the primary dataset was 0.902 ± 0.024 (95% CI (confidence interval), 0.873–0.932). In the independent validation, the mean AUC value for all prediction models was 0.747 ± 0.034 (95% CI, 0.705–0.790)."

"In conclusion, they investigated the feasibility of a framework for predicting malignant glioma grades based on radiomics using CE-T1WIs and T2WIs. The authors proposed framework could sufficiently and easily predict malignant glioma grades by preparing images acquired by a few structural MRI sequences."

"The proposed framework with a few MRI sequences could mitigate the tedious process of tumor contouring on each MRI sequence image compared with the frameworks with multiple MRI sequences. In addition, the best prediction performances of this study indicated that their proposed framework with a few MRI sequences could have versatility to varied datasets. Their proposed framework for noninvasively grading malignant gliomas based on the preoperative images could be an effective tool for selection of appropriate surgery and educating the patients."

https://www.nature.com/articles/s41598-019-55922-0

In [ ]:
#Code by Md. Redwan Karim Sony https://www.kaggle.com/redwankarimsony/visualizing-and-analyzing-dicoms-in-python/notebook?select=train.csv

print('Brain Segmentation Mask')
imageio.mimsave(f'./{sample_id}.gif', segmented_brains_fill, duration=0.1)
display.Image(f'./{sample_id}.gif', format='png')

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Prediction of Malignant Glioma using MRI</span></h1><br>

Citation: Nakamoto, T., Takahashi, W., Haga, A. et al. Prediction of malignant glioma grades using contrast-enhanced T1-weighted and T2-weighted magnetic resonance images based on a radiomic analysis. Sci Rep 9, 19411 (2019).

https://doi.org/10.1038/s41598-019-55922-0

![](https://media.springernature.com/m685/springer-static/image/art%3A10.1038%2Fs41598-019-55922-0/MediaObjects/41598_2019_55922_Fig1_HTML.png)nature.com

https://www.nature.com/articles/s41598-019-55922-0

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Imaging Microglia and Astrocytes using Diffusion MRI</span></h1><br>

Imaging Microglia and Astrocytes non-invasively using Diffusion MRI

Authors: Raquel Garcia-Hernandez, Alejandro Trouve Carpena, Mark Drakesmith, Kristen Koller, Derek K. Jones, Santiago Canals, Silvia De Santis

doi: https://doi.org/10.1101/2020.02.07.938910

"The authors demonstrate that the MRI signal carries the fingerprint of microglia and astrocytes activation, and that specific signatures from each glia population can be extracted in vivo."

<font color="#0000FF">Data analysis and statistics</font>

"The statistical analysis was done using GraphPad Prism 7 software and Rstudio. The presence of outlier values and parametric distribution was checked. In most of the analysis the authors applied t-test comparing, for each time point, control hemisphere versus injected hemisphere."

"For regression analysis, Pearson’s correlation was applied and coefficients were transformed to apply Fisher’s 1925 test for significant values. Polar analysis and dispersion estimation was performed to obtain kappa values, before transforming them into an Orientation Dispersion Index (ODI)."

"Conclusion: the authors proposed there a new generation of non-invasive glia-centric biomarkers, which are expected to transform the study of many diseases associated with a glial response: those where inflammation is as a known or possible cause, as well as those in which the glial reaction can serve as a powerful early diagnostic and/or prognostic marker."

https://www.biorxiv.org/content/10.1101/2020.02.07.938910v1.full

In [ ]:
#Code by Md. Redwan Karim Sony https://www.kaggle.com/redwankarimsony/visualizing-and-analyzing-dicoms-in-python/notebook?select=train.csv

print('Segmented Part of Brain Tissue')
imageio.mimsave(f'./{sample_id}.gif', seg_brain_pixels, duration=0.1)
display.Image(f'./{sample_id}.gif', format='png')

In [ ]:
#Code by Md. Redwan Karim Sony https://www.kaggle.com/redwankarimsony/visualizing-and-analyzing-dicoms-in-python/notebook?select=train.csv

from skimage.morphology import opening, closing
from skimage.morphology import disk

def plot_comparison(original, filtered, filter_name):

    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 4), sharex=True,
                                   sharey=True)
    ax1.imshow(original, cmap=plt.cm.gray)
    ax1.set_title('original')
    ax1.axis('off')
    ax2.imshow(filtered, cmap=plt.cm.gray)
    ax2.set_title(filter_name)
    ax2.axis('off')

In [ ]:
#Code by Md. Redwan Karim Sony https://www.kaggle.com/redwankarimsony/visualizing-and-analyzing-dicoms-in-python/notebook?select=train.csv

original = segmented_brains_fill[65]

rows = 4
cols = 4
f, ax = plt.subplots(rows, cols, figsize = (15,12))

for i in range(rows*cols):
    if i==0:
        ax[0,0].imshow(original, cmap = plt.cm.gray)
        ax[0,0].set_title('Original')
        ax[0,0].axis(False)
    else:
        closed = closing(original, disk(i))
        ax[int(i/rows),int(i % rows)].set_title(f'closed disk({i})')
        ax[int(i/rows),int(i % rows)].imshow(closed, cmap = plt.cm.gray)
        ax[int(i/rows),int(i % rows)].axis('off')
plt.show()

#Above, I failed again! Hence, I hid input/outputs.

We can select the desired filters from here and just multiply that with the original image to get brain segmentation from these. Before selecting any particular filter, let's see the segmentation quality of the filters and then we can select the desired filter.

In [ ]:
#Code by Md. Redwan Karim Sony https://www.kaggle.com/redwankarimsony/visualizing-and-analyzing-dicoms-in-python/notebook?select=train.csv

original_image = sample_pixels[65]
original = segmented_brains_fill[65]
f, ax = plt.subplots(rows, cols, figsize = (10,10))

for i in range(rows*cols):
    if i==0:
        ax[0,0].imshow(original_image, cmap = plt.cm.gray)
        ax[0,0].set_title('Original')
        ax[0,0].axis(False)
    else:
        closed = closing(original, disk(i))
        ax[int(i/rows),int(i % rows)].set_title(f'closed disk({i})')
        ax[int(i/rows),int(i % rows)].imshow(original_image * closed, cmap = plt.cm.gray)
        ax[int(i/rows),int(i % rows)].axis('off')
plt.show() 

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Differentiation between Glioblastoma and Solitary Metastasis</span></h1><br>

Differentiation between Glioblastoma and Solitary Metastasis: Morphologic Assessment by Conventional Brain MR Imaging and Diffusion-Weighted Imaging

Authors: Bo Young Jung, Eun Ja Lee, Jong Myon Bae, Young Jae Choi, Eun Kyoung Lee, and Dae Bong Kim

Investig Magn Reson Imaging. 2021 Mar;25(1):23-34. English.
Published online Mar 31, 2021.  https://doi.org/10.13104/imri.2021.25.1.23

"Differentiating between glioblastoma and solitary metastasis is very important for the planning of further workup and treatment. The authors assessed the ability of various morphological parameters using conventional MRI and diffusion-based techniques to distinguish between glioblastomas and solitary metastases in tumoral and peritumoral regions."

Statistical Analysis

"The authors did statistical analyses using the SPSS software package (ver. 20.0). To find out whether there were significant differences between glioblastoma and metastasis, they analyzed the morphologic parameters of the enhancing tumor and peritumoral regions using the chi-squared test. They also evaluated which sequence was best for depicting peritumoral neoplastic cell infiltration. Furthermore, they used receiver operating characteristic (ROC) analysis to assess the extent of the peritumoral non-enhancing lesion and to find the cutoff value with the optimum combination of sensitivity and specificity for distinguishing glioblastoma from metastasis; P < 0.05 was taken to indicate statistical significance."

"This study had several limitations. Biopsy of peritumoral non-enhancing lesions was not done as part of the histologic examination at the time of surgery. A limited number of patients. Because of the small sample, factors that were less useful for differentiating glioblastoma from metastasis may not have been included."

"Depending on the isocitrate dehydrogenase (IDH1) mutation status, 1p19q codeletion and methylation status of O-6-methylguanine-DNA methyltransferase (MGMT), glioblastoma has had different treatment successes, different patient populations, and different imaging findings in both conventional and advanced MR imaging."

"In their study, patient's diagnoses with glioblastoma were based only on histological phenotypes, because they were diagnosed before WHO 2016 revision."

<font color="#DC143C">"Therefore, the authors failed to evaluate the different imaging findings of glioblastoma based on genetic mutations and molecular markers."</font>


"In conclusion, they found that visual assessment of tumoral and peritumoral regions, using conventional MRI and diffusion-based techniques, allows them to distinguish these two types of tumor."

https://i-mri.org/DOIx.php?id=10.13104/imri.2021.25.1.23

In [ ]:
df=pd.read_csv('/kaggle/input/cusersmarildownloadstumorcsv/tumor.csv',encoding ='ISO-8859-1',sep=";")
df.head(10)

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Glioblastoma</span></h1><br>

"Glioblastoma (GBM) accounts for 40%∼50% of all primary malignant brain tumors in adults. Brain metastases are the most common complication of systemic cancer, and half of them are solitary at diagnosis. It is clinically important to distinguish GBM from solitary brain metastasis (SBM) because of the vast differences in these 2 entities with regard to tumor staging, treatment approach, and clinical outcomes. Structural gadolinium-enhanced MR imaging is the preferred imaging examination for brain tumors, but with a limited capacity to differentiate GBM (Glioblastoma) and SBM (solitary brain metastasis). They share similar imaging features, such as extensive edema and ring-enhancement, which is a great challenge in clinical practice."

http://www.ajnr.org/content/early/2020/03/05/ajnr.A6466

In [ ]:
#By Glioblastoma

glioblastoma = pd.DataFrame( {     
"age" : ["60 ± 10.05"], 
"sex_perc_of_men)" : ['57,1'],    
"irregular_shape" : ["18/21 (85.7%)"] ,
"round_or_ovoid_shape" : ["3/21 (14.3%)"],
"Ill_defined_margin" : ["16/21 (76.2%)"], 
"well_defined_margin" : ["5/21 (23.8%)"],
"heterogeneous_enhancement" : ["16/21 (76.2%)"], 
"homogeneous_enhancement" : ["5/21 (23.8%)"],
"presence_of_hemorrhage" : ["13/14 (92.9%)"],
"absence_of_hemorrhage" : ["1/14 (7.1%)"],} ) 
glioblastoma

"Glioblastomas had an irregular shape, ill-defined margins, and a heterogeneous enhancement pattern; on the other hand, metastases had an ovoid or round shape, well-defined margins, and homogeneous enhancement. Metastases had significantly more extensive peritumoral T2 high signal intensity than glioblastomas had. In visual assessment of peritumoral neoplastic cell infiltration using T2-weighted and diffusion-based images, all sequences differed significantly between the two groups. Exponential DWI (Diffusion-weighted images) had the highest sensitivity for the diagnosis of both glioblastoma (100%) and metastasis (70.6%). A combination of exponential DWI and ADC maps (Attenuation Coefficient (AC) map.) was optimal for the depiction of peritumoral neoplastic cell infiltration in glioblastoma."

Conclusion

"In the differentiation of glioblastoma from solitary metastatic lesions, visual morphologic assessment of tumoral and peritumoral regions using conventional MRI and diffusion-based techniques can also offer diagnostic information."

https://i-mri.org/DOIx.php?id=10.13104/imri.2021.25.1.23

#Thanks to Md. Redwan Karim Sony for all the Dicom script. Give support to his work.

https://www.kaggle.com/redwankarimsony/visualizing-and-analyzing-dicoms-in-python/notebook?select=train.csv

#Thanks to Jose Carmona for the Slider with ipywidgets. 
